In [1]:
# import data
import pandas as pd
import numpy as np

colnames = ['CIC0', 'SM1_Dz(Z)', 'GATS1i', 'NdsCH', 'NdssC','MLOGP', 'LC50']
data = pd.read_csv('./data/qsar_fish_toxicity.csv', sep=';', names=colnames)
X = data.drop('LC50', axis=1)
y = data['LC50']

# Print the sizes of the dataset
print("Number of Examples in the Dataset = ", X.shape[0])
print("Number of Features for each example = ", X.shape[1])
# print output range
print("Output Range = [%f, %f]" %(min(y), max(y)))

Number of Examples in the Dataset =  908
Number of Features for each example =  6
Output Range = [0.053000, 9.612000]


In [2]:
# Defne the Keras models
from keras.models import Sequential
from keras.layers import Dense

def build_model_1():
    # build the Keras model_1
    model = Sequential()
    model.add(Dense(4, input_dim=X.shape[1], activation='relu'))
    model.add(Dense(1))
    # Compile the model
    model.compile(loss='mean_squared_error', optimizer='adam')
    # return the model
    return model

def build_model_2():
    # build the Keras model_2
    model = Sequential()
    model.add(Dense(8, input_dim=X.shape[1], activation='relu'))
    model.add(Dense(1))
    # Compile the model
    model.compile(loss='mean_squared_error', optimizer='adam')
    # return the model
    return model

def build_model_3():
    # build the Keras model_3
    model = Sequential()
    model.add(Dense(4, input_dim=X.shape[1], activation='relu'))
    model.add(Dense(2, activation='relu'))
    model.add(Dense(1))
    # Compile the model
    model.compile(loss='mean_squared_error', optimizer='adam')
    # return the model
    return model

Using TensorFlow backend.


In [3]:
# define a seed for random number generator so the result will be reproducible
from tensorflow import random 
seed = 1
np.random.seed(seed)
random.set_seed(seed)

# perform cross validation on each model
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
results_1 = []
models = [build_model_1, build_model_2, build_model_3]
# loop over three models
for m in range(len(models)):
    model = KerasRegressor(build_fn=models[m], epochs=100, batch_size=20, verbose=0, shuffle=False)
    kf = KFold(n_splits=3)
    result = cross_val_score(model, X, y, cv=kf)
    results_1.append(result)

In [4]:
# print the cross validation scores
print("Cross Validation Loss for Model 1 =", abs(results_1[0].mean()))
print("Cross Validation Loss for Model 2 =", abs(results_1[1].mean()))
print("Cross Validation Loss for Model 3 =", abs(results_1[2].mean()))

Cross Validation Loss for Model 1 = 1.0369543234507244
Cross Validation Loss for Model 2 = 0.9512714942296346
Cross Validation Loss for Model 3 = 2.9471448262532554


In [5]:
# define a seed for random number generator so the result will be reproducible
np.random.seed(seed)
random.set_seed(seed)
results_2 = []
epochs = [100, 150]
batches = [20, 15]

# Loop over pairs of epochs and batch_size
for e in range(len(epochs)):
    for b in range(len(batches)):
        model = KerasRegressor(build_fn= build_model_2, epochs= epochs[e], batch_size= batches[b], verbose=0, shuffle=False)
        kf = KFold(n_splits=3)
        result = cross_val_score(model, X, y, cv=kf)
        results_2.append(result)

In [6]:
# Print cross validation score for each possible pair of epochs, batch_size
c = 0
for e in range(len(epochs)):
    for b in range(len(batches)):
        print("batch_size =", batches[b],", epochs =", epochs[e], ", Test Loss =", abs(results_2[c].mean()))
        c += 1

batch_size = 20 , epochs = 100 , Test Loss = 0.982687771320343
batch_size = 15 , epochs = 100 , Test Loss = 0.9392397006352743
batch_size = 20 , epochs = 150 , Test Loss = 0.9431027173995972
batch_size = 15 , epochs = 150 , Test Loss = 0.9389869173367819


In [7]:
# Modify build_model_2 function
def build_model_2(activation='relu', optimizer='adam'):
    # build the Keras model_2
    model = Sequential()
    model.add(Dense(8, input_dim=X.shape[1], activation=activation))
    model.add(Dense(1))
    # Compile the model
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    # return the model
    return model

results_3 = []
activations = ['relu', 'tanh']
optimizers = ['sgd', 'adam', 'rmsprop']

# define a seed for random number generator so the result will be reproducible
np.random.seed(seed)
random.set_seed(seed)
# Loop over pairs of activation and optimizer
for o in range(len(optimizers)):
    for a in range(len(activations)):
        optimizer = optimizers[o]
        activation = activations[a]
        model = KerasRegressor(build_fn= build_model_2, epochs=100, batch_size=20, verbose=0, shuffle=False)
        kf = KFold(n_splits=3)
        result = cross_val_score(model, X, y, cv=kf)
        results_3.append(result)

In [8]:
# Print cross validation score for each possible pair of optimizer, activation
c = 0
for o in range(len(optimizers)):
    for a in range(len(activations)):
        print("activation = ", activations[a],", optimizer = ", optimizers[o], ", Test Loss = ", abs(results_3[c].mean()))
        c += 1

activation =  relu , optimizer =  sgd , Test Loss =  0.982687771320343
activation =  tanh , optimizer =  sgd , Test Loss =  0.9512714942296346
activation =  relu , optimizer =  adam , Test Loss =  0.9581676125526428
activation =  tanh , optimizer =  adam , Test Loss =  0.9816934863726298
activation =  relu , optimizer =  rmsprop , Test Loss =  1.0462734301884968
activation =  tanh , optimizer =  rmsprop , Test Loss =  1.0267855525016785
